In [1]:
from bpch2nc import bpch_2_netcdf
import numpy as np
import xarray as xr
import re

In [2]:
case_name = 'CO2-2018'
first_day   = '2018-02-01'
last_day_p1 = '2018-03-01'

In [3]:
# Name of Bpch file
directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'

name_bpch1 = 'ts_satellite.'
tinfo_file = directory + 'tracerinfo.dat'
dinfo_file = directory + 'diaginfo.dat'

# Output
output_directory = '/geos/u73/msadiq/GEOS-Chem/rundirs/ensemble_runs/' + case_name + '/nd51/'

# Number of seconds in the diagnostic interval (assume 1-month)
# does not matter for CO2
interval = 86400.0 * 31.0

In [4]:
days = np.arange(first_day, last_day_p1, dtype='datetime64[D]')

for iday in np.arange(len(days)):
    day_string = days[iday] # format not right for the following function
    print('converting bpch to netcdf on day: ', day_string)
    new_day_string = re.sub("[^0-9]", "", str(day_string)) # strip off '-'s
    
    bpchfile = directory + name_bpch1 + new_day_string + '.bpch'
    ncfile = output_directory + name_bpch1 + new_day_string + '.nc'
    
    bpch_2_netcdf(bpchfile=bpchfile, 
                  tinfo_file=tinfo_file, 
                  dinfo_file=dinfo_file, 
                  ncfile=ncfile)

converting bpch to netcdf on day:  2018-02-01
converting bpch to netcdf on day:  2018-02-02
converting bpch to netcdf on day:  2018-02-03
converting bpch to netcdf on day:  2018-02-04
converting bpch to netcdf on day:  2018-02-05
converting bpch to netcdf on day:  2018-02-06
converting bpch to netcdf on day:  2018-02-07
converting bpch to netcdf on day:  2018-02-08
converting bpch to netcdf on day:  2018-02-09
converting bpch to netcdf on day:  2018-02-10
converting bpch to netcdf on day:  2018-02-11
converting bpch to netcdf on day:  2018-02-12
converting bpch to netcdf on day:  2018-02-13
converting bpch to netcdf on day:  2018-02-14
converting bpch to netcdf on day:  2018-02-15
converting bpch to netcdf on day:  2018-02-16
converting bpch to netcdf on day:  2018-02-17
converting bpch to netcdf on day:  2018-02-18
converting bpch to netcdf on day:  2018-02-19
converting bpch to netcdf on day:  2018-02-20
converting bpch to netcdf on day:  2018-02-21
converting bpch to netcdf on day: 

In [5]:
# prepare output data format
new_day_string = re.sub("[^0-9]", "", str(first_day)) # strip off '-'s
first_file = xr.open_dataset(directory + name_bpch1 + new_day_string + '.nc')
varnames = list(first_file.data_vars.keys())  # a list of variable names
lon = first_file.lon
lat = first_file.lat
lev = first_file.lev
time = days
target = xr.DataArray(np.nan, coords=[time, lev, lat, lon], dims=['time', 'lev', 'lat', 'lon'])
output = target.to_dataset(name = 'SpeciesConc_CO2')
output.attrs = first_file.attrs
for ivar in varnames: output[ivar] = target.copy()


In [6]:
# combine the netcdf files into one, monthly
for iday in np.arange(len(days)):
    day_string = days[iday] #
    print(day_string)
    new_day_string = re.sub("[^0-9]", "", str(day_string)) # strip off '-'s
    ncfile = output_directory + name_bpch1 + new_day_string + '.nc'
    
    ds_tmp = xr.open_dataset(ncfile)
    
    for ivar in varnames:
        output[ivar][iday,:,:,:] = ds_tmp[ivar][0,:,:,:].copy()
        output[ivar].attrs = ds_tmp[ivar].attrs

2018-02-01
2018-02-02
2018-02-03
2018-02-04
2018-02-05
2018-02-06
2018-02-07
2018-02-08
2018-02-09
2018-02-10
2018-02-11
2018-02-12
2018-02-13
2018-02-14
2018-02-15
2018-02-16
2018-02-17
2018-02-18
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-24
2018-02-25
2018-02-26
2018-02-27
2018-02-28


In [7]:
# output monthly combined data into a NetCDF file
first_day_string = re.sub("[^0-9]", "", str(first_day)) # strip off '-'s
monthly_string = first_day_string[0:6]
output.to_netcdf(output_directory + name_bpch1 + monthly_string + '.nc')

In [8]:
# flatten the 4d output
ds = xr.open_dataset(output_directory + name_bpch1 + monthly_string + '.nc')
ds['SpeciesConc_CO2']

<xarray.DataArray 'SpeciesConc_CO2' (time: 28, lev: 47, lat: 91, lon: 144)>
[17244864 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2018-02-01 2018-02-02 ... 2018-02-28
  * lev      (lev) float64 0.9925 0.9775 0.9624 ... 0.0003988 0.000127 2.763e-05
  * lat      (lat) float64 -89.5 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
  * lon      (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
Attributes:
    units:      ppbv
    long_name:  CO2 tracer

In [9]:
nm_lev = len(ds.lev)
nm_lon = len(ds.lon)
nm_lat = len(ds.lat)
nm_days = len(ds.time)

record = range(nm_lon * nm_lat * nm_days)
record

range(0, 366912)

In [10]:
target = xr.DataArray(np.nan, coords=[record, ds.lev], dims=['record', 'levels'])
ds_output = target.to_dataset(name = 'SpeciesConc_CO2')
for ivar in varnames: ds_output[ivar] = target.copy()
ds_output

<xarray.Dataset>
Dimensions:          (levels: 47, record: 366912)
Coordinates:
  * record           (record) int64 0 1 2 3 4 ... 366908 366909 366910 366911
  * levels           (levels) float64 0.9925 0.9775 ... 0.000127 2.763e-05
Data variables:
    SpeciesConc_CO2  (record, levels) float64 nan nan nan nan ... nan nan nan
    PEDGE_S_PSURF    (record, levels) float64 nan nan nan nan ... nan nan nan

In [11]:
flat = ds['SpeciesConc_CO2'][:,0,:,:].to_dataframe() # flatten a variable at one level
flat.reset_index(inplace=True) # get indices to prepare output coordinates
flat

,time,lat,lon,lev,SpeciesConc_CO2
0,2018-02-01,-89.5,-180.0,0.9925,403494.09375
1,2018-02-01,-89.5,-177.5,0.9925,403491.37500
2,2018-02-01,-89.5,-175.0,0.9925,403488.28125
3,2018-02-01,-89.5,-172.5,0.9925,403486.00000
4,2018-02-01,-89.5,-170.0,0.9925,403483.34375
...,...,...,...,...,...
366907,2018-02-28,89.5,167.5,0.9925,415165.84375
366908,2018-02-28,89.5,170.0,0.9925,415163.09375
366909,2018-02-28,89.5,172.5,0.9925,415162.15625
366910,2018-02-28,89.5,175.0,0.9925,415160.28125


In [12]:
lat = xr.DataArray(0, coords=[record], dims=['record'])
lon = xr.DataArray(0, coords=[record], dims=['record'])
date = xr.DataArray(0, coords=[record], dims=['record'])
lat.values = flat['lat']
lon.values = flat['lon']
date.values = flat['time']
ds_output['lat'] = lat
ds_output['lon'] = lon
ds_output['date'] = date
ds_output

<xarray.Dataset>
Dimensions:          (levels: 47, record: 366912)
Coordinates:
  * record           (record) int64 0 1 2 3 4 ... 366908 366909 366910 366911
  * levels           (levels) float64 0.9925 0.9775 ... 0.000127 2.763e-05
Data variables:
    SpeciesConc_CO2  (record, levels) float64 nan nan nan nan ... nan nan nan
    PEDGE_S_PSURF    (record, levels) float64 nan nan nan nan ... nan nan nan
    lat              (record) float64 -89.5 -89.5 -89.5 -89.5 ... 89.5 89.5 89.5
    lon              (record) float64 -180.0 -177.5 -175.0 ... 172.5 175.0 177.5
    date             (record) datetime64[ns] 2018-02-01 ... 2018-02-28

In [13]:
for ivar in varnames:
    target = xr.DataArray(np.nan, coords=[record, ds.lev], dims=['record', 'levels'])
    for ilev in range(nm_lev):
        flat = ds[ivar][:,ilev,:,:].to_dataframe() # flatten a variable at one level
        target[:,ilev] = flat[ivar] # store output to a dataarray
    ds_output[ivar] = target.copy() # store dataarray to dataset
    print(ivar + ' done!')

SpeciesConc_CO2 done!
PEDGE_S_PSURF done!


In [14]:
# 
ds_output.attrs['comment'] = '[CO2] unit: ppbv'
ds_output.record.values

array([     0,      1,      2, ..., 366909, 366910, 366911])

In [15]:
output_file = '2d_' + name_bpch1 + monthly_string + '.nc'
ds_output.to_netcdf(output_directory + output_file)
print(output_file)

2d_ts_satellite.201802.nc
